# Simulator Wrapper

In [1]:
!pip install pyNetLogo
!pip install JPype1
!pip install GPy==1.10.0
!pip install GPyOpt==1.2.1
!pip install emukit

In [1]:
import numpy as np

Here you should specify the dorectory where you install the NetLogo. 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')

import pyNetLogo

netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home=r"D:\Programs\NetLogo 6.2.0", netlogo_version="6.2")

Here you should specify the path of the Netlogo model you have downloaded in order to load the model.

In [3]:
netlogo.load_model(r'C:\Users\Chengzu\Downloads\Blood_Sugar_Regulation.nlogo')

In [4]:
problem = {
        'num_vars': 4,
        'names': [
                  'metabolic-rate',
                  'insulin-sensitivity',
                  'glucose-sensitivity',
                  'glucagon-sensitivity',
                  ],
        'bounds': [[0, 200],
                   [0., 1.],
                   [0., 1.],
                   [0., 1.]]
    }

In [5]:
def run_simulation(experiment, start_time_step=90, end_time_step=100):
    """ run a netlogo model
    Parameters
    ----------
    experiments : dict
    """

    # Set the input parameters
    
    for key, value in experiment.items():
        if key == 'random-seed':
            # The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            # Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(key, value))

    netlogo.command('setup')
    # Run for 100 ticks and return the number of glucose
    counts = netlogo.repeat_report(['count glucoses', 'count insulins', 'count glucagons'], 101)

    results = pd.Series([counts['count glucoses'].values[start_time_step: end_time_step].mean()],
                        index=['Avg. glucoses'])
    return results

# Load Specified Parameters

In [6]:
def specify_parameters(file: str, variable_X: list):
    '''
    The file should be a .csv file, with who, metabolic-rate, insulin-sensitivity, glucose-sensitivity, glucagon-sensitivity as its title.
    The returned params are a list full of lists
    '''
    params = []
    import pandas as pd
    df = pd.read_csv(file)
    for i in range(len(df)):
        item = df.iloc[i].to_dict()
        _ = item.pop("who")
        item_values = []
        for k, v in item.items():
            if k not in variable_X:
                continue
            else:
                if k == "metabolic-rate":
                    v = v/200
                item_values.append(v)
        params.append(item_values)
    return params

# Initialize Simulator

In [7]:
class Simulator:
    def __init__(self, variable_X):
        self.variable_X = variable_X
        
    def simulate(self, input_values):
        assert len(input_values[0]) == len(self.variable_X)
        item = {}
        for k, v in zip(self.variable_X, input_values[0]):
            if k == "metabolic-rate":
                v = v * 200
            item[k] = v
        result = run_simulation(item)
        return np.array([result['Avg. glucoses']]).reshape(-1, 1)
    
    def opt_simulate(self, input_values):
        assert len(input_values[0]) == len(self.variable_X)
        item = {}
        for k, v in zip(self.variable_X, input_values[0]):
            if k == "metabolic-rate":
                v = v * 200
            item[k] = v
        result = run_simulation(item)
        return np.array([np.abs(result['Avg. glucoses'] - 4000)]).reshape(-1, 1)

# Gaussian Process as emulator

In [8]:
import GPy

In [9]:
kernel_list = ["RBF", "Matern52", "Matern32", "Exponential", "OU", "RatQuad"]
# https://scikit-learn.org/stable/modules/gaussian_process.html#gp-kernels
# https://gpy.readthedocs.io/en/deploy/GPy.kern.src.html#module-GPy.kern.src.stationary

def get_kernel(kernel_name: str, input_dim: int, length_scale: float, variance: float):
    # squared-exponential
    if kernel_name == "RBF":
        return GPy.kern.RBF(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    # Matern covariance
    elif kernel_name == "Matern52":
        return GPy.kern.Matern52(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "Matern32":
        return GPy.kern.Matern32(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    # # periodic covariance
    elif kernel_name == "Periodic":
        return GPy.kern.Periodic(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "Exponential":
        return GPy.kern.Exponential(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "OU":
        return GPy.kern.OU(input_dim=input_dim, variance=variance, lengthscale=length_scale)
    elif kernel_name == "RatQuad":
        return GPy.kern.RatQuad(input_dim=input_dim, variance=variance, lengthscale=length_scale)

    else:
        raise KeyError("Unsupported kernel here.")

In [10]:
def emulate(method: str, X: list, Y: list, kernel_name: str, length_scale: float, variance: float, noise_var: float = 1e-10, verbose: bool = False):
    if method == "GP":
        model = Gaussian_Process(X, Y, kernel_name, length_scale, variance, verbose, noise_var)
        return model
    else:
        raise ValueError("Unsupported Emulator.")

In [11]:
def Gaussian_Process(X: list, Y: list, kernel_name: str, length_scale: float, variance: float, verbose: bool = False, noise_var: float = 1e-10):
    if verbose:
        print(f"Using {kernel_name} kernel and Gaussian Process as Emulator.")
    input_dim = len(X[0])
    print('input_dim', input_dim)
    kernel = get_kernel(kernel_name, input_dim, length_scale, variance)
    model = GPy.models.GPRegression(np.array(X), np.array(Y)[:,np.newaxis], kernel, noise_var=noise_var) # noise_var=1e-10
    _ = model.optimize(messages=verbose) # Optimize parameters of covariance function
    print(_)
    print(f"{kernel_name}-log_likelihood", model.log_likelihood())
    return model

# Experiments

In [12]:
variable_X = ["metabolic-rate", "insulin-sensitivity", "glucose-sensitivity", "glucagon-sensitivity"]
simulator = Simulator(variable_X)

In [13]:
X = specify_parameters(
    file=r"../my_sample_data.csv",
    variable_X=variable_X
)

In [14]:
Y = [simulator.simulate([p]).reshape(-1)[0] for p in X]

In [15]:
assert len(Y) == len(X)

In [17]:
kernel_n="RBF"
length_scale=1
variance=1

Y_select = np.abs(np.array(Y) - 4000)
model = emulate(
      method="GP",
      X=X,
      Y=Y_select,
      kernel_name=kernel_n,
      length_scale=length_scale,
      variance=variance
    )

input_dim 4
Optimizer: 				 L-BFGS-B (Scipy implementation)
f(x_opt): 				 80.940
Number of function evaluations: 	 87
Optimization status: 			 Converged
Time elapsed: 				 0:00:00.110751

RBF-log_likelihood -80.94024488406353
